# Analysis notebook comparing scoping vs no-scoping for tower selection
Purpose of this notebook is to categorize and analyze generated towers.

Requires:
* `.pkl` generated by `stimuli/score_towers.py`

See also:
* `stimuli/generate_towers.ipynb` for plotting code and a similar analysis in the same place as the tower generation code. This notebook supersedes it.

In [5]:
# set up imports
import os
import sys
__file__ = os.getcwd()
proj_dir =  os.path.dirname(os.path.realpath(__file__))
sys.path.append(proj_dir)
utils_dir = os.path.join(proj_dir,'utils')
sys.path.append(utils_dir)
analysis_dir = os.path.join(proj_dir,'analysis')
analysis_utils_dir = os.path.join(analysis_dir,'utils')
sys.path.append(analysis_utils_dir)
agent_dir = os.path.join(proj_dir,'model')
sys.path.append(agent_dir)
agent_util_dir = os.path.join(agent_dir,'utils')
sys.path.append(agent_util_dir)
experiments_dir = os.path.join(proj_dir,'experiments')
sys.path.append(experiments_dir)
df_dir = os.path.join(proj_dir,'results/dataframes')
stim_dir = os.path.join(proj_dir,'stimuli')

In [7]:
import tqdm

import pickle

import math

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

from utils.blockworld_library import *
from utils.blockworld import *

from model.BFS_Lookahead_Agent import BFS_Lookahead_Agent
from model.BFS_Agent import BFS_Agent
from model.Astar_Agent import Astar_Agent

pygame 2.1.0 (SDL 2.0.16, Python 3.9.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [8]:
# some helper functions

# look at towers
def visualize_towers(towers, text_parameters=None):
    fig, axes = plt.subplots(math.ceil(len(towers)/5),
                             5, figsize=(20, 15*math.ceil(len(towers)/20)))
    for axis, tower in zip(axes.flatten(), towers):
        axis.imshow(tower['bitmap']*1.0)
        if text_parameters is not None:
            if type(text_parameters) is not list:
                text_parameters = [text_parameters]
            for y_offset, text_parameter in enumerate(text_parameters):
                axis.text(0, y_offset*1., str(text_parameter+": " +
                          str(tower[text_parameter])), color='gray', fontsize=20)
    plt.tight_layout()
    plt.show()


Load in data

In [9]:
path_to_df = os.path.join(df_dir,"scoring towers.pkl")
df = pd.read_pickle(path_to_df)

In [10]:
df

,run_ID,agent,world,step,planning_step,states_evaluated,action,_action,action_x,action_block_width,...,all_sequences_planning_cost,decomposed_silhouette,_all_subgoal_sequences,_chosen_subgoal_sequence,F1,final_row,perfect,agent_attributes_string,avg_cost_per_step_for_run,agent_label
0,1 | Best_First_Search_Agent heuristic: F1score...,Best_First_Search_Agent,1.0,0.0,1.0,NaN,"[(3x1), 3]","((3x1), 3)",3.0,3.0,...,NaN,NaN,NaN,NaN,0.240000,False,False,"{'agent_type': 'Best_First_Search_Agent', 'heu...",NaN,Best_First_Search_Agent
1,1 | Best_First_Search_Agent heuristic: F1score...,Best_First_Search_Agent,1.0,1.0,1.0,NaN,"[(3x1), 3]","((3x1), 3)",3.0,3.0,...,NaN,NaN,NaN,NaN,0.428571,False,False,"{'agent_type': 'Best_First_Search_Agent', 'heu...",NaN,Best_First_Search_Agent
2,1 | Best_First_Search_Agent heuristic: F1score...,Best_First_Search_Agent,1.0,2.0,1.0,NaN,"[(3x1), 3]","((3x1), 3)",3.0,3.0,...,NaN,NaN,NaN,NaN,0.580645,False,False,"{'agent_type': 'Best_First_Search_Agent', 'heu...",NaN,Best_First_Search_Agent
3,1 | Best_First_Search_Agent heuristic: F1score...,Best_First_Search_Agent,1.0,3.0,1.0,NaN,"[(2x1), 3]","((2x1), 3)",3.0,2.0,...,NaN,NaN,NaN,NaN,0.666667,False,False,"{'agent_type': 'Best_First_Search_Agent', 'heu...",NaN,Best_First_Search_Agent
4,1 | Best_First_Search_Agent heuristic: F1score...,Best_First_Search_Agent,1.0,4.0,1.0,NaN,"[(2x1), 4]","((2x1), 4)",4.0,2.0,...,NaN,NaN,NaN,NaN,0.742857,False,False,"{'agent_type': 'Best_First_Search_Agent', 'heu...",NaN,Best_First_Search_Agent
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
160,"9 | {'agent_type': 'Subgoal_Planning_Agent', '...",Subgoal_Planning_Agent,9.0,6.0,1.0,NaN,"[(1x2), 3]","((1x2), 3)",3.0,1.0,...,NaN,NaN,NaN,NaN,0.790698,False,False,"{'agent_type': 'Subgoal_Planning_Agent', 'sequ...",NaN,Subgoal_Planning_Agent
161,"9 | {'agent_type': 'Subgoal_Planning_Agent', '...",Subgoal_Planning_Agent,9.0,7.0,1.0,NaN,"[(1x2), 3]","((1x2), 3)",3.0,1.0,...,NaN,NaN,NaN,NaN,0.844444,False,False,"{'agent_type': 'Subgoal_Planning_Agent', 'sequ...",NaN,Subgoal_Planning_Agent
162,"9 | {'agent_type': 'Subgoal_Planning_Agent', '...",Subgoal_Planning_Agent,9.0,8.0,1.0,NaN,"[(1x3), 4]","((1x3), 4)",4.0,1.0,...,NaN,NaN,NaN,NaN,0.916667,False,False,"{'agent_type': 'Subgoal_Planning_Agent', 'sequ...",NaN,Subgoal_Planning_Agent
163,"9 | {'agent_type': 'Subgoal_Planning_Agent', '...",Subgoal_Planning_Agent,9.0,9.0,1.0,NaN,"[(2x1), 4]","((2x1), 4)",4.0,2.0,...,NaN,NaN,NaN,NaN,0.960000,False,False,"{'agent_type': 'Subgoal_Planning_Agent', 'sequ...",NaN,Subgoal_Planning_Agent


In [14]:
df.columns

Index(['run_ID', 'agent', 'world', 'step', 'planning_step', 'states_evaluated',
       'action', '_action', 'action_x', 'action_block_width',
       'action_block_height', 'blocks', '_blocks', 'blockmap', '_world',
       'legal_action_space', 'fast_failure', 'execution_time', 'world_status',
       'world_failure_reason', 'agent_attributes', 'agent_type', 'heuristic',
       'random_seed', 'sequence_length', 'decomposition_function',
       'include_subsequences', 'number_of_sequences', 'c_weight', 'max_cost',
       'step_size', 'lower level: agent_type', 'lower level: heuristic',
       'lower level: random_seed', 'partial_solution_cost', 'solution_cost',
       'partial_planning_cost', 'planning_cost', 'all_sequences_planning_cost',
       'decomposed_silhouette', '_all_subgoal_sequences',
       '_chosen_subgoal_sequence', 'F1', 'final_row', 'perfect',
       'agent_attributes_string', 'avg_cost_per_step_for_run', 'agent_label'],
      dtype='object')

In [12]:
# extract final rows
df_final = df[df['final_row'] == True]

In [13]:
df_final

,run_ID,agent,world,step,planning_step,states_evaluated,action,_action,action_x,action_block_width,...,all_sequences_planning_cost,decomposed_silhouette,_all_subgoal_sequences,_chosen_subgoal_sequence,F1,final_row,perfect,agent_attributes_string,avg_cost_per_step_for_run,agent_label
8,1 | Best_First_Search_Agent heuristic: F1score...,Best_First_Search_Agent,1.0,8.0,1.0,202.0,"[(1x2), 4]","((1x2), 4)",4.0,1.0,...,NaN,NaN,NaN,NaN,1.0,True,True,"{'agent_type': 'Best_First_Search_Agent', 'heu...",22.444444,Best_First_Search_Agent
16,2 | Best_First_Search_Agent heuristic: F1score...,Best_First_Search_Agent,2.0,7.0,1.0,1831.0,"[(3x1), 3]","((3x1), 3)",3.0,3.0,...,NaN,NaN,NaN,NaN,1.0,True,True,"{'agent_type': 'Best_First_Search_Agent', 'heu...",228.875000,Best_First_Search_Agent
26,3 | Best_First_Search_Agent heuristic: F1score...,Best_First_Search_Agent,3.0,9.0,1.0,20208.0,"[(1x2), 2]","((1x2), 2)",2.0,1.0,...,NaN,NaN,NaN,NaN,1.0,True,True,"{'agent_type': 'Best_First_Search_Agent', 'heu...",2020.800000,Best_First_Search_Agent
35,0 | Best_First_Search_Agent heuristic: F1score...,Best_First_Search_Agent,0.0,8.0,1.0,34749.0,"[(1x2), 4]","((1x2), 4)",4.0,1.0,...,NaN,NaN,NaN,NaN,1.0,True,True,"{'agent_type': 'Best_First_Search_Agent', 'heu...",3861.000000,Best_First_Search_Agent
43,5 | Best_First_Search_Agent heuristic: F1score...,Best_First_Search_Agent,5.0,7.0,1.0,15.0,"[(2x1), 3]","((2x1), 3)",3.0,2.0,...,NaN,NaN,NaN,NaN,1.0,True,True,"{'agent_type': 'Best_First_Search_Agent', 'heu...",1.875000,Best_First_Search_Agent
51,6 | Best_First_Search_Agent heuristic: F1score...,Best_First_Search_Agent,6.0,7.0,1.0,26.0,"[(2x1), 2]","((2x1), 2)",2.0,2.0,...,NaN,NaN,NaN,NaN,1.0,True,True,"{'agent_type': 'Best_First_Search_Agent', 'heu...",3.250000,Best_First_Search_Agent
61,7 | Best_First_Search_Agent heuristic: F1score...,Best_First_Search_Agent,7.0,9.0,1.0,747.0,"[(2x1), 2]","((2x1), 2)",2.0,2.0,...,NaN,NaN,NaN,NaN,1.0,True,True,"{'agent_type': 'Best_First_Search_Agent', 'heu...",74.700000,Best_First_Search_Agent
71,9 | Best_First_Search_Agent heuristic: F1score...,Best_First_Search_Agent,9.0,9.0,1.0,65446.0,"[(3x1), 3]","((3x1), 3)",3.0,3.0,...,NaN,NaN,NaN,NaN,1.0,True,True,"{'agent_type': 'Best_First_Search_Agent', 'heu...",6544.600000,Best_First_Search_Agent
81,8 | Best_First_Search_Agent heuristic: F1score...,Best_First_Search_Agent,8.0,9.0,1.0,196599.0,"[(1x2), 3]","((1x2), 3)",3.0,1.0,...,NaN,NaN,NaN,NaN,1.0,True,True,"{'agent_type': 'Best_First_Search_Agent', 'heu...",19659.900000,Best_First_Search_Agent
89,"5 | {'agent_type': 'Subgoal_Planning_Agent', '...",Subgoal_Planning_Agent,5.0,7.0,1.0,NaN,"[(2x1), 3]","((2x1), 3)",3.0,2.0,...,36079.0,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0...",[[<model.utils.decomposition_functions.Subgoal...,[<model.utils.decomposition_functions.Subgoal_...,1.0,True,True,"{'agent_type': 'Subgoal_Planning_Agent', 'sequ...",0.000000,Subgoal_Planning_Agent


Let's explore the data a little bit

df.groupby('agent')